<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Develop-Data-Transformation-Functions" data-toc-modified-id="Develop-Data-Transformation-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Develop Data Transformation Functions</a></span></li></ul></div>

This notebooks contains the development of the transformation pipeline / module for the trx data. The final code is run in `transform_data.py`.


Flags:
-    Flag purchase with redemption
-    Flag purchase without redemption when v_sum = 0
-    Flag puchase without redemption when v_sum > 0
-    Flag purchase by device
-    Flag purchase with discount, considering a threshold
-   _Flag purchase with value < 0 as return (not implemented)_

Intervals:
-   purchase intervals
-   interval from activation to next purchase (not considering activation)

In [1]:
import datetime as dt
import itertools
import sys
from pathlib import Path
from pytest import approx

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.feather as feather
import seaborn as sns

import codebook.EDA as EDA
import codebook.clean as clean

In [2]:
print(sys.executable)
print(sys.version)
print(f"Pandas Version: {pd.__version__}")
print(f"Seaborn Version: {sns.__version__}")

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
Pandas Version: 1.1.3
Seaborn Version: 0.11.0


In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('precision', 2)
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [4]:
# Load from feather file
data_clean = feather.read_feather("data/1_trx_data_clean.feather")

In [5]:
data_clean.head(2)
data_clean.info()

,member,date,trx_type,device,value,discount
0,178,2018-01-10,Purchase,Payment,83.1,60.8
1,178,2018-01-19,Purchase,Payment,146.3,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625348 entries, 0 to 1625347
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype         
---  ------    --------------    -----         
 0   member    1625348 non-null  int64         
 1   date      1625348 non-null  datetime64[ns]
 2   trx_type  1625348 non-null  object        
 3   device    1625348 non-null  object        
 4   value     1625348 non-null  float32       
 5   discount  1625348 non-null  float32       
dtypes: datetime64[ns](1), float32(2), int64(1), object(2)
memory usage: 62.0+ MB


In [6]:
data_clean["member"] = data_clean["member"].astype("object")
data_clean["device"] = data_clean["device"].astype("category")
data_clean["trx_type"] = data_clean["trx_type"].astype("category")

In [7]:
# Sort by trx_type to have Activation --> Purchase --> Redemption on the same date
data_clean.sort_values(["member", "date", "trx_type"], inplace=True)

## Develop Data Transformation Functions

**Note:** 
- There are a few same day activations / redemptions, I have handled that.
- There are redemptions without purchase transaction - I do not consider them as purchase!

In [8]:
# Define a test_member (tm)
tm = data_clean[data_clean["member"] == 199189].copy()
tm.reset_index(drop=True, inplace=True)
tm

,member,date,trx_type,device,value,discount
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00
1,199189,2018-05-12,Redemption,Financial Voucher,-10.00,0.00
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00
3,199189,2018-08-11,Purchase,Loyalty,20.00,49.90
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00
5,199189,2018-10-05,Activation,Financial Voucher,5.00,0.00
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00
7,199189,2018-10-22,Redemption,Financial Voucher,-5.00,0.00
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00


In [9]:
# Add an edge case: Same-date Activation and redemption
test = tm.copy()
test.iloc[5, 1] = dt.datetime.strptime("2018-10-22", "%Y-%m-%d")
test.iloc[3, 3] = "Payment"  # just that I not forget about these values
test

,member,date,trx_type,device,value,discount
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00
1,199189,2018-05-12,Redemption,Financial Voucher,-10.00,0.00
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00
3,199189,2018-08-11,Purchase,Payment,20.00,49.90
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00
7,199189,2018-10-22,Redemption,Financial Voucher,-5.00,0.00
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00


In [10]:
# # Alternative customer with a redmeption without purchase
# test = data_clean[data_clean["member"] == 249039].copy()

In [11]:
# This can be applied to whole df in one go

def create_basic_voucher_cols(df):
    """Create separate columns containing the values for 
    voucher activations ("v_a"), voucher redemptions ("v_r")
    and all of them combined.
    """
    df = df.copy()
    df["v_a"] = np.where(
        (df["device"] == "Financial Voucher") & (df["value"] > 0),
        df["value"], 
        0)
    df["v_r"] = np.where(
        (df["device"] == "Financial Voucher") & (df["value"] < 0),
        df["value"],
        0)
    df["v"] = np.where(
        df["device"] == "Financial Voucher",
        df["value"],
        0)
    return df

In [12]:
test = create_basic_voucher_cols(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,0.0,0.0
1,199189,2018-05-12,Redemption,Financial Voucher,-10.00,0.00,0.0,-10.0,-10.0
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,0.0,0.0
7,199189,2018-10-22,Redemption,Financial Voucher,-5.00,0.00,0.0,-5.0,-5.0
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0


In [13]:
def _calculate_voucher_sums(v):
    """Helper function to calculate the accumulated sum of
    voucher "credit" a customer has at any given time. We cannot 
    kow about credit from earlier periods but we make sure that
    the sum never becomes negative.
    """
    v_sum = np.array(list(itertools.accumulate(v)))
    # Make sure that v_sum never has a negative value
    v_min = np.min(v_sum)
    if v_min < 0:
        top_up_value = v_min
        v_sum = v_sum - top_up_value
    return v_sum

def create_voucher_sum_col(df):
    """Use a groupby "window function" to insert the
    voucher sums into a new column "v_sum".
    """
    df = df.copy()
    df = df.assign(v_sum = df.groupby(
        ["member"])["v"].transform(_calculate_voucher_sums))
    df.drop("v", axis=1, inplace=True)
    return df
    

(Check this stackoverflow post for [apply vs. transform on groupby objects](https://stackoverflow.com/questions/27517425/apply-vs-transform-on-a-group-object))

In [14]:
test = create_voucher_sum_col(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,0.0,10.0
1,199189,2018-05-12,Redemption,Financial Voucher,-10.00,0.00,0.0,-10.0,0.0
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,0.0,5.0
7,199189,2018-10-22,Redemption,Financial Voucher,-5.00,0.00,0.0,-5.0,0.0
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0


In [15]:
def shift_and_drop_redemptions(df):
    """Shift redemtion values in "v_r" column one row up, so they
    end up in the row of the corresponing transaction. This makes
    it possible to flag the respective transactions as ones with 
    redemption in a later step. Then delete all redemption rows, as 
    they are no longer needed.
    """
    df = df.copy()
    df = df.assign(v_r = df.groupby(
        ["member"])["v_r"].shift(-1)
    )
    df = df[~df["trx_type"].isin(["Redemption"])]
    data_trans["v_r"] = data_trans["v_r"].replace(np.nan, 0)
    
    # Remove "Redemption" Category
    df["trx_type"].cat.remove_unused_categories(inplace=True)  
    return df

In [16]:
test = shift_and_drop_redemptions(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,-10.0,10.0
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,-5.0,5.0
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0
10,199189,2019-04-25,Purchase,Loyalty,246.90,61.70,0.0,0.0,0.0
11,199189,2019-06-05,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0


In [17]:
def calculate_interval_activation_to_next_purchase(df):
    """Create a new col "delta_a" containing the interval from each
    activation to the next purchase as int (for days). Values of
    all non-activation rows are set to NaN.
    """
    df = df.copy()
    df = df.assign(delta_a=df.groupby(
        ["member"])["date"].diff(-1) * -1)
    
    df["delta_a"] = np.where(
        df["trx_type"] == "Activation", 
        df["delta_a"].dt.days,
        np.NaN
    )
    return df

In [18]:
def calculate_purchase_interval(df):
    """Create a new col "delta_p" containing the interval from each
    purchase to the next as int (for days). Values of all non-purchase
    rows are set to NaN.
    """
    df = df.copy()
    df = df.assign(delta_p=df.groupby(
        ["member", "trx_type"])["date"].diff(-1) * -1)
    
    df["delta_p"] = np.where(
        df["trx_type"] == "Purchase", 
        df["delta_p"].dt.days,
        np.NaN
    )
    return df

In [19]:
test = calculate_interval_activation_to_next_purchase(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum,delta_a
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,-10.0,10.0,NaN
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0,NaN
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0,NaN
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0,NaN
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,0.0
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,-5.0,5.0,NaN
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0,NaN
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0,NaN
10,199189,2019-04-25,Purchase,Loyalty,246.90,61.70,0.0,0.0,0.0,NaN
11,199189,2019-06-05,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,94.0


In [20]:
test = calculate_purchase_interval(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum,delta_a,delta_p
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,-10.0,10.0,NaN,58.0
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0,NaN,33.0
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0,NaN,20.0
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0,NaN,52.0
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,0.0,NaN
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,-5.0,5.0,NaN,73.0
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0,NaN,81.0
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0,NaN,31.0
10,199189,2019-04-25,Purchase,Loyalty,246.90,61.70,0.0,0.0,0.0,NaN,135.0
11,199189,2019-06-05,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,94.0,NaN


In [21]:
def flag_purchases_depending_on_vouchers(df):
    """Create three new boolean columns to classify purchases into
    each of the following three categories: "p_v_red" = purchase with
    redemption, "p_v_miss" = purchase without redemption (but voucher
    credit would have been available), "p_v_empty" = no voucher credit
    available.
    """
    df = df.copy()
    df["p_v_red"] = np.where(
        (df["trx_type"] == "Purchase") & (df["v_r"] < 0), 1, 0
    ).astype("bool")
    df["p_v_miss"] = np.where(
        (df["trx_type"] == "Purchase") & (df["v_r"] == 0) & (df["v_sum"] > 0), 1, 0
    ).astype("bool")
    df["p_v_empty"] = np.where(
        (df["trx_type"] == "Purchase") & (df["v_sum"] == 0), 1, 0
    ).astype("bool")
    return df

In [22]:
test = flag_purchases_depending_on_vouchers(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum,delta_a,delta_p,p_v_red,p_v_miss,p_v_empty
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,-10.0,10.0,NaN,58.0,True,False,False
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0,NaN,33.0,False,False,True
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0,NaN,20.0,False,False,True
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0,NaN,52.0,False,False,True
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,0.0,NaN,False,False,False
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,-5.0,5.0,NaN,73.0,True,False,False
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0,NaN,81.0,False,False,True
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0,NaN,31.0,False,False,True
10,199189,2019-04-25,Purchase,Loyalty,246.90,61.70,0.0,0.0,0.0,NaN,135.0,False,False,True
11,199189,2019-06-05,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,94.0,NaN,False,False,False


In [23]:
def calculate_discount_pct(df):
    """Caclulate a column "discount_pct" denoting the relative
    value of discounts. This value will be used to control for a
    threshold when setting a discount flag in the next step. (Note
    the calculation is such that discounts on returns wont reach
    the threshold.)
    """
    df = df.copy()
    df["gross_value"] = df["value"] + df["discount"]
    df["discount_pct"] = df["discount"] / df["gross_value"]
    return df


def flag_purchases_depending_on_discounts(df, threshold_pct=0.1):
    """Create a boolean columns to classify purchases having a
    discount whose relative value to the gross transaction price 
    reaches a certain threshold.
    """
    df = df.copy()
    df["p_discount"] = np.where(
        df["discount_pct"] >= threshold_pct, 1, 0
    ).astype("bool")
    
    df.drop(["gross_value", "discount_pct"], axis=1, inplace=True)
    return df

In [24]:
test = calculate_discount_pct(test)
test = flag_purchases_depending_on_discounts(test, threshold_pct=0.1)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum,delta_a,delta_p,p_v_red,p_v_miss,p_v_empty,p_discount
0,199189,2018-05-12,Purchase,Loyalty,107.70,0.00,0.0,-10.0,10.0,NaN,58.0,True,False,False,False
2,199189,2018-07-09,Purchase,Loyalty,127.70,0.00,0.0,0.0,0.0,NaN,33.0,False,False,True,False
3,199189,2018-08-11,Purchase,Payment,20.00,49.90,0.0,0.0,0.0,NaN,20.0,False,False,True,True
4,199189,2018-08-31,Purchase,Loyalty,31.80,8.00,0.0,0.0,0.0,NaN,52.0,False,False,True,True
5,199189,2018-10-22,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,0.0,NaN,False,False,False,False
6,199189,2018-10-22,Purchase,Loyalty,49.80,0.00,0.0,-5.0,5.0,NaN,73.0,True,False,False,False
8,199189,2019-01-03,Purchase,Loyalty,8.95,8.95,0.0,0.0,0.0,NaN,81.0,False,False,True,True
9,199189,2019-03-25,Purchase,Loyalty,44.90,0.00,0.0,0.0,0.0,NaN,31.0,False,False,True,False
10,199189,2019-04-25,Purchase,Loyalty,246.90,61.70,0.0,0.0,0.0,NaN,135.0,False,False,True,True
11,199189,2019-06-05,Activation,Financial Voucher,5.00,0.00,5.0,0.0,5.0,94.0,NaN,False,False,False,False


In [25]:
# Not implemented - just an idea

def prettify_remaining_cols(df):
    """Finally transform the trx_type into a boolean column
    for purchase / non purchase. And just to save some space,
    shorten the string for vouchers in the device column.
    """
    df = df.copy()
    df["trx_type"] = np.where(df["trx_type"] == "Purchase", 1, 0).astype("bool")
    df["device"] = np.where(df["device"] == "Financial Voucher", "Voucher", df["device"])
    return df

In [26]:
test = prettify_remaining_cols(test)
test

,member,date,trx_type,device,value,discount,v_a,v_r,v_sum,delta_a,delta_p,p_v_red,p_v_miss,p_v_empty,p_discount
0,199189,2018-05-12,True,Loyalty,107.70,0.00,0.0,-10.0,10.0,NaN,58.0,True,False,False,False
2,199189,2018-07-09,True,Loyalty,127.70,0.00,0.0,0.0,0.0,NaN,33.0,False,False,True,False
3,199189,2018-08-11,True,Payment,20.00,49.90,0.0,0.0,0.0,NaN,20.0,False,False,True,True
4,199189,2018-08-31,True,Loyalty,31.80,8.00,0.0,0.0,0.0,NaN,52.0,False,False,True,True
5,199189,2018-10-22,False,Voucher,5.00,0.00,5.0,0.0,5.0,0.0,NaN,False,False,False,False
6,199189,2018-10-22,True,Loyalty,49.80,0.00,0.0,-5.0,5.0,NaN,73.0,True,False,False,False
8,199189,2019-01-03,True,Loyalty,8.95,8.95,0.0,0.0,0.0,NaN,81.0,False,False,True,True
9,199189,2019-03-25,True,Loyalty,44.90,0.00,0.0,0.0,0.0,NaN,31.0,False,False,True,False
10,199189,2019-04-25,True,Loyalty,246.90,61.70,0.0,0.0,0.0,NaN,135.0,False,False,True,True
11,199189,2019-06-05,False,Voucher,5.00,0.00,5.0,0.0,5.0,94.0,NaN,False,False,False,False


---